Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


**You can't just copy** from the lesson notebook to this assignment.

- Because the lesson was **regression**, but the assignment is **classification.**
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [2]:
import category_encoders as cat
import numpy as np
from scipy import stats

from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import RandomizedSearchCV, train_test_split, cross_val_score
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.preprocessing import StandardScaler, FunctionTransformer

In [3]:
import pandas as pd
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 100

# Merge train_features.csv & train_labels.csv
df = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv')).set_index('id')

# Read test_features.csv & sample_submission.csv
df_test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv', index_col='id')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

In [4]:
target = 'status_group'
X_train = df.drop(columns=target)
y_train = df[target]

In [5]:
X_train.describe(exclude='number').T.sort_values(by='unique').T

,recorded_by,public_meeting,permit,source_class,management_group,quantity_group,quantity,waterpoint_type_group,quality_group,payment_type,source_type,waterpoint_type,extraction_type_class,payment,water_quality,basin,source,scheme_management,management,extraction_type_group,extraction_type,region,lga,date_recorded,funder,ward,installer,scheme_name,subvillage,wpt_name
count,59400,56066,56344,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,55523,59400,59400,59400,59400,59400,59400,55765,59400,55745,31234,59029,59400
unique,1,2,2,3,5,5,5,6,6,7,7,7,7,7,8,9,10,12,12,13,18,21,125,356,1897,2092,2145,2696,19287,37400
top,GeoData Consultants Ltd,True,True,groundwater,user-group,enough,enough,communal standpipe,good,never pay,spring,communal standpipe,gravity,never pay,soft,Lake Victoria,spring,VWC,vwc,gravity,gravity,Iringa,Njombe,2011-03-15,Government Of Tanzania,Igosi,DWE,K,Madukani,none
freq,59400,51011,38852,45794,52490,33186,33186,34625,50818,25348,17021,28522,26780,25348,50818,10248,17021,36793,40507,26780,26780,5294,2503,572,9084,307,17402,682,508,3563


In [6]:
def wrangle(df):
    df = df.copy()
    
    date_recorded = pd.to_datetime(df['date_recorded'])
    
    df['month_recorded'] = date_recorded.dt.month
    df['year_recorded'] = date_recorded.dt.year
    
    ### recorded_by has only 1 unique, so no information
    ### subvillage and wpt_name are VERY high cardinality
    ### permit is least related to the target, as determined by chi2 tests
    
    df = df.drop(columns=['recorded_by', 'subvillage', 'wpt_name', 'permit'])
            
    return df

In [7]:
X_train = wrangle(X_train)

### Split features into categories, to use in our Great Transformer

num_feats = X_train.select_dtypes('number').columns
cat_feats = X_train.select_dtypes('object').columns
cat_feats_low_card = [f for f in cat_feats if X_train[f].nunique() < 100]
cat_feats_high_card = [f for f in cat_feats if X_train[f].nunique() >= 100]

In [8]:
### Used in the Great Transformer. Sets NaNs to the NaN count in that column.

def CountImputer(df):
    for col in range(df.shape[1]):
        df[:, col] = np.nan_to_num(df[:, col], nan=np.count_nonzero(np.isnan(df[:, col])))
    return df

In [9]:
### THE GREAT TRANSFORMER

### This is a tree of its own sort, using FeatureUnions, Pipelines, 
### and ColumnTransformers to split the set by certain categories, 
### perform different encodings, and re-merge.

transformers = FeatureUnion([ 
    ('pca_part', Pipeline([              ## This section OneHot encodes low card columns, Count encodes
        ('encoders_for_pca',             ## both low and high card columns, and performs PCA on the aggregate
         ColumnTransformer([             ## keeping only the first principal component
            ('low_card', FeatureUnion([
                ('onehotencoder', cat.OneHotEncoder(use_cat_names=True)),
                ('countencoder', cat.CountEncoder(min_group_size=.1))
            ]), cat_feats_low_card),
            ('high_card', cat.CountEncoder(min_group_size=.1), cat_feats_high_card)
        ], verbose=True)),
        ('realimputer',
         FunctionTransformer(CountImputer)),
        ('standardscaler', 
         StandardScaler()),
        ('pca', 
         PCA(n_components=1))
    ])),
    ('ordinal_pipe',
     ColumnTransformer([                                           ## This section corresponds to the basic
         ('ordinalencoder', cat.OrdinalEncoder(), cat_feats),      ## ordinal encoding
         ('dumb', FunctionTransformer(lambda df: df), num_feats)
     ]))
])

In [10]:
transformers.fit(X_train)

c:\users\benjs\opencv\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


[ColumnTransformer] ...... (1 of 2) Processing low_card, total=   4.0s
[ColumnTransformer] ..... (2 of 2) Processing high_card, total=   0.1s


FeatureUnion(transformer_list=[('pca_part',
                                Pipeline(steps=[('encoders_for_pca',
                                                 ColumnTransformer(transformers=[('low_card',
                                                                                  FeatureUnion(transformer_list=[('onehotencoder',
                                                                                                                  OneHotEncoder(use_cat_names=True)),
                                                                                                                 ('countencoder',
                                                                                                                  CountEncoder(combine_min_nan_groups=True,
                                                                                                                               min_group_size=0.1))]),
                                                                           

In [11]:
XT_train = pd.DataFrame(transformers.transform(X_train), index=X_train.index)
XT_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
69572,-4.588200,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,6000.0,1390.0,34.938093,-9.856322,0.0,11.0,5.0,109.0,1999.0,3.0,2011.0
8776,-2.513710,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,0.0,1399.0,34.698766,-2.147466,0.0,20.0,2.0,280.0,2010.0,3.0,2013.0
34310,-3.800448,3.0,3.0,3.0,3.0,3.0,3.0,3.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,1.0,1.0,1.0,1.0,3.0,3.0,2.0,2.0,1.0,25.0,686.0,37.460664,-3.821329,0.0,21.0,4.0,250.0,2009.0,2.0,2013.0
67743,2.036770,4.0,4.0,4.0,4.0,4.0,4.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,3.0,3.0,4.0,4.0,1.0,2.0,1.0,0.0,263.0,38.486161,-11.155298,0.0,90.0,63.0,58.0,1986.0,1.0,2013.0
19728,-3.125814,5.0,5.0,5.0,2.0,5.0,5.0,5.0,1.0,3.0,2.0,1.0,1.0,1.0,3.0,2.0,2.0,2.0,1.0,1.0,4.0,4.0,2.0,2.0,2.0,1.0,1.0,0.0,0.0,31.130847,-1.825359,0.0,18.0,1.0,0.0,0.0,7.0,2011.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60739,-4.922434,58.0,59.0,50.0,3.0,11.0,64.0,651.0,1.0,6.0,317.0,1.0,1.0,1.0,5.0,1.0,3.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,10.0,1210.0,37.169807,-3.253847,0.0,3.0,5.0,125.0,1999.0,5.0,2013.0
27263,-5.771579,318.0,423.0,1036.0,8.0,1.0,16.0,641.0,1.0,1.0,697.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,7.0,7.0,2.0,1.0,1.0,4700.0,1212.0,35.249991,-9.070629,0.0,11.0,4.0,56.0,1996.0,5.0,2011.0
37057,3.474072,75.0,26.0,20.0,8.0,18.0,45.0,450.0,1.0,1.0,2.0,3.0,3.0,3.0,1.0,1.0,7.0,7.0,5.0,5.0,1.0,1.0,4.0,4.0,1.0,3.0,2.0,0.0,0.0,34.017087,-8.750434,0.0,12.0,7.0,0.0,0.0,4.0,2011.0


In [12]:
X_test = wrangle(df_test)

In [13]:
XT_test = pd.DataFrame(transformers.transform(X_test), index=X_test.index)

In [14]:
model = RandomForestClassifier(n_estimators=1200, 
                               random_state=42, 
                               n_jobs=-1, 
                               max_depth=30, 
                               max_leaf_nodes=4500,
                               min_samples_leaf=2,
                               min_samples_split=3)

In [15]:
model.fit(XT_train, y_train)

RandomForestClassifier(max_depth=30, max_leaf_nodes=4500, min_samples_leaf=2,
                       min_samples_split=3, n_estimators=1200, n_jobs=-1,
                       random_state=42)

In [16]:
predictions = pd.DataFrame({'status_group': model.predict(XT_test)}, index=XT_test.index)
predictions.to_csv('pump_submission_comp1.csv')